In [1]:
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import skimage.transform as st
import numpy as np
import h5py
import io
import math

In [2]:
def retreiveImages(hf, file_path):

    img_names = hf['Anno_fine'][file_path]
    img_names = np.array(img_names)
    strip = np.vectorize(lambda x : x.strip())
    img_names = strip(img_names)
    
    imgs = []
    for img_name in tqdm(img_names):
        img = np.asarray(Image.open(io.BytesIO(np.array(hf[img_name]))))


        imgs.append(img)

    return imgs

In [3]:
def getSequence(hf, file_name):
    ret = hf['Anno_fine'][file_name]
    ret = np.array(ret)
    ret = [[int(y) for y in x.strip().split()] for x in ret]
    return np.array(ret)

In [4]:
def preprocess(index, imgs, bboxs, landmarks):
    img = imgs[index]

    target_size = 301

    y_diff = target_size - img.shape[0]
    x_diff = target_size - img.shape[1]

    x_diff_left = math.floor(x_diff/2)
    x_diff_right = math.ceil(x_diff/2)

    y_diff_up = math.floor(y_diff/2)
    y_diff_down = math.ceil(y_diff/2)

    left_padding = np.max(img) * np.ones((img.shape[0], x_diff_left, 3), dtype=np.uint8)
    right_padding = np.max(img) * np.ones((img.shape[0], x_diff_right, 3), dtype=np.uint8)

    new_img = np.hstack((left_padding, img, right_padding))

    up_padding = np.max(img) * np.ones((y_diff_up, target_size, 3), dtype=np.uint8)
    down_padding = np.max(img) * np.ones((y_diff_down, target_size, 3), dtype=np.uint8)

    new_img = np.vstack((up_padding, new_img, down_padding))
    
    i = Image.fromarray(new_img)
    i = i.resize(size=(224, 224))

    imgs[index] = np.array(i)
    bboxs[index] = [sum(x) for x in zip(train_bboxs[index], [x_diff_left, y_diff_up] * 2)]
    landmarks[index] = [sum(x) for x in zip(train_landmarks[index], [x_diff_left, y_diff_up] * 8)]


In [5]:
hf = h5py.File('../data/data.h5', 'r')

train_imgs = retreiveImages(hf, 'train.txt')
train_attrs = getSequence(hf, 'train_attr.txt')
train_categories = getSequence(hf, 'train_cate.txt')
train_bboxs = getSequence(hf, 'train_bbox.txt')
train_landmarks = getSequence(hf, 'train_landmarks.txt')

for i in tqdm(range(len(train_imgs))):
    preprocess(i, train_imgs, train_bboxs, train_landmarks)

hf.close()

In [6]:
hf = h5py.File('../data/train_fine.h5', 'w')
hf.create_dataset('img', data=np.array(train_imgs))
hf.create_dataset('attr', data=train_attrs)
hf.create_dataset('category', data=train_categories)
hf.create_dataset('bbox', data=train_bboxs)
hf.create_dataset('landmark', data=train_landmarks)
hf.close()

In [7]:
hf = h5py.File('../data/data.h5', 'r')

train_imgs = retreiveImages(hf, 'test.txt')
train_attrs = getSequence(hf, 'test_attr.txt')
train_categories = getSequence(hf, 'test_cate.txt')
train_bboxs = getSequence(hf, 'test_bbox.txt')
train_landmarks = getSequence(hf, 'test_landmarks.txt')

for i in tqdm(range(len(train_imgs))):
    preprocess(i, train_imgs, train_bboxs, train_landmarks)

hf.close()

In [8]:
hf = h5py.File('../data/test_fine.h5', 'w')
hf.create_dataset('img', data=np.array(train_imgs))
hf.create_dataset('attr', data=train_attrs)
hf.create_dataset('category', data=train_categories)
hf.create_dataset('bbox', data=train_bboxs)
hf.create_dataset('landmark', data=train_landmarks)
hf.close()